In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import constants as const
import db_operations as dbop
import data_prepare as dp
import ml_model
import customized_obj as cus_obj

import xgboost.sklearn as xgb
import lightgbm.sklearn as lgbm
import sklearn.preprocessing as preproc
import sklearn.metrics as metrics
import sklearn as sk

import datetime
import time
import multiprocessing as mp

In [15]:
db_type = "sqlite3"
conn = dbop.connect_db(db_type)

cols_category = ["area", "industry", "market", "exchange", "is_hs"]
df_stock_basic = dbop.create_df(conn.cursor(), const.STOCK_BASIC[const.TABLE])
# print(df_stock_basic[df_stock_basic[cols_category].isna().any(axis=1)][
#           cols_category])
df_stock_basic.loc[:, cols_category] = df_stock_basic[cols_category].fillna("")

enc = sk.preprocessing.OrdinalEncoder()
val_enc = enc.fit_transform(df_stock_basic[cols_category])
df_stock_basic.loc[:, cols_category] = val_enc
print("basic:", np.max(val_enc))
print(val_enc[np.isnan(val_enc)])
print(np.sum(np.isnan(val_enc)))

# print(df_stock_basic.dtypes)
print(df_stock_basic[cols_category].dtypes)
# print(df_stock_basic[cols_category].iloc[:50])
print(df_stock_basic[df_stock_basic[cols_category].isnull().any(axis=1)])

if np.max(val_enc)<2**8:
#     df_stock_basic.loc[:,cols_category] = df_stock_basic[cols_category].astype("uint8")
    df_stock_basic[cols_category] = df_stock_basic[cols_category].astype("uint8")
    print(df_stock_basic.dtypes)

basic: 110.0
[]
0
area        object
industry    object
market      object
exchange    object
is_hs       object
dtype: object
Empty DataFrame
Columns: [code, symbol, name, area, industry, fullname, enname, market, exchange, curr_type, list_status, list_date, delist_date, is_hs]
Index: []
code           object
symbol         object
name           object
area            uint8
industry        uint8
fullname       object
enname         object
market          uint8
exchange        uint8
curr_type      object
list_status    object
list_date      object
delist_date    object
is_hs           uint8
dtype: object


In [16]:
X = df_stock_basic[cols_category]
y = df_stock_basic[cols_category].sum(axis=1)
print(X.shape,y.shape)

(3669, 5) (3669,)


In [17]:
reg = lgbm.LGBMRegressor(n_estimators=10)
print(reg.fit(X,y))

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=10,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
